In [90]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

# Вхідний файл, який містить дані
input_file = 'income_data.txt'

In [91]:
# Читання даних
X = []
y = []
count_class1 = 0
count_class2 = 0
max_datapoints = 25000


In [92]:
with open(input_file, 'r') as f:
  for line in f:
    if count_class1 >= max_datapoints and count_class2 >= max_datapoints:
      break
    if '?' in line:
      continue

    data = line.strip().split(',')

    if data[-1].strip() == '<=50K' and count_class1 < max_datapoints:
      X.append(data)
      count_class1 += 1
    elif data[-1].strip() == '>50K' and count_class2 < max_datapoints:
      X.append(data)
      count_class2 += 1

In [93]:
# Перетворення на масив numpy
X = np.array(X)

# Перетворення рядкових даних на числові
label_encoder = []
X_encoded = np.empty(X.shape)

for i,item in enumerate(X[1]):
 if item.isdigit():
  X_encoded[:, i] = X[:, i]
 else:
  label_encoder.append(preprocessing.LabelEncoder())
  X_encoded[:, i] = label_encoder[-1].fit_transform(X[:,i])
X = X_encoded[:, :-1].astype(int)
y = X_encoded[:, -1].astype(int)

In [94]:
# Створення SVМ-класифікатора
classifier = OneVsOneClassifier(LinearSVC(random_state=0))

In [95]:
# Навчання класифікатора
classifier.fit(X, y)

OneVsOneClassifier(estimator=LinearSVC(random_state=0))

In [96]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [97]:
classifier = OneVsOneClassifier(LinearSVC(random_state=0))
classifier.fit(X_train, y_train)
y_test_pred = classifier.predict(X_test)

In [98]:
# Обчислення F-міри для SVМ-класифікатора
f1 = cross_val_score(classifier, X, y, scoring='f1_weighted', cv=3)
print("F1 score: " + str(round(100*f1.mean(), 2)) + "%")

F1 score: 74.26%


In [100]:
# Передбачення результату для тестової точки даних
input_data = ['37', 'Private', '215646', 'HS-grad', '9', 'Nevermarried', 'Handlers-cleaners', 'Not-in-family', 'White', 'Male',
'0', '0', '40', 'United-States']

In [107]:
input_data_encoded = [-1] * len(input_data)
count = 0

for i, item in enumerate(input_data):
    if item.isdigit():
        input_data_encoded[i] = int(item)
    else:
        try:
            input_data_encoded[i] = int(label_encoder[count].transform([item])[0])
        except ValueError:
            # Якщо значення не зустрічалося раніше, присвоюємо значення -1 або інше значення для "private"\"unknown"
            input_data_encoded[i] = -1
        count += 1

input_data_encoded = np.array(input_data_encoded)

In [109]:
# Використання класифікатора для кодованої точки даних
# та виведення результату
predicted_class = classifier.predict(input_data_encoded.reshape(1, -1))
print(label_encoder[-1].inverse_transform(predicted_class)[0])


 <=50K


In [116]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Передбачаємо результати для тестової вибірки
y_pred = classifier.predict(X_test)

# Обчислюємо метрики
accuracy = round(accuracy_score(y_test, y_pred), 2)
precision = round(precision_score(y_test, y_pred, average='weighted'), 2)
recall = round(recall_score(y_test, y_pred, average='weighted'), 2)
f1 = round(f1_score(y_test, y_pred, average='weighted'), 2)

# Виведення результатів
print(f"Точність (accuracy): {accuracy}%")
print(f"Прецизійність (precision): {precision}%")
print(f"Повнота (recall): {recall}%")
print(f"F1-score: {f1}%")


Точність (accuracy): 0.78%
Прецизійність (precision): 0.76%
Повнота (recall): 0.78%
F1-score: 0.74%
